<a href="https://colab.research.google.com/github/RaminParker/bundesamt_fuer_statistik/blob/main/Bundesamt_f%C3%BCr_Statistik_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install feedparser

In [ ]:
import feedparser
import re
import datetime
from datetime import timedelta
import requests
from pathlib import Path

**Genesis API credentials**

get yours for free if you haven't already
https://www-genesis.destatis.de/genesis/online?Menu=Registrierung

In [ ]:
# Mount your Google Drive on your Google Colab runtime
from google.colab import drive
drive.mount('/content/drive/')

path_to_key = "/content/drive/MyDrive/Colab Notebooks/configs/configs_statistisches_bundesamt/passwort.txt" 
file1 = open(path_to_key, "r")
passwort = file1.read()

path_to_username = "/content/drive/MyDrive/Colab Notebooks/configs/configs_statistisches_bundesamt/kennung.txt"
file2 = open(path_to_username, "r")
kennung = file2.read()

In [ ]:
# insert your credentials here before you proceed
# Genesis Online, Regionaldatenbank and Zensusdatenbank have each their own registration and usernames
KENNUNG = kennung
PASSWORT = passwort

In [ ]:
# api login test - check credentials first
checkUrl = "https://www-genesis.destatis.de/genesisWS/rest/2020/helloworld/logincheck?username=" \
            +KENNUNG+ "&password=" +PASSWORT

response = requests.get(checkUrl, timeout=120)
response.json()

In [ ]:
# Latest updates via Genesis rss feed
feed = feedparser.parse("https://www-genesis.destatis.de/genesis/online/news?language=de")

In [ ]:
today = datetime.datetime.now()
today

# today = datetime.datetime(2022, 1, 2) # yyyy m d
# today

In [ ]:
days_past = 1
yesterday = (today - timedelta(days=days_past)).strftime("%d.%m.%Y")
yesterday

In [ ]:
# create and populate list of updated statistics from newsfeed
updatedStatistics = []

for entry in feed.entries:
    
    # filter feed to new items of the last 2 days
    if (today - datetime.datetime(*entry.published_parsed[:4])).days < days_past:
    
        myCode = re.findall(r"\d{5}",entry.title)[0] # parse table code
        updatedStatistics.append(myCode)
        
        print(myCode, entry.published, entry.title[17:85])

In [ ]:
len(updatedStatistics)

In [ ]:
# create and populate list of tables that belong to each updated statistic
updatedTables = []

for statistic in updatedStatistics:
    catUrl = "https://www-genesis.destatis.de/genesisWS/rest/2020/catalogue/tables2statistic?username=" \
                +KENNUNG+"&password="+PASSWORT+"&name="+statistic
    catResponse = requests.get(catUrl, timeout=120)

    for code in catResponse.json()["List"]:
        updatedTables.append(code["Code"])

print(updatedTables)

In [ ]:
konsumausgaben = ["81000", "63221"]
einkommen = ["12211", "21421", "22151", "61351", "62361", "63121", "63221", "13311", "62321", "62361", "13111"]
einzelhandel = ["12211", "13111", "21211", "23611", "45212", "45341", "45211", "45212"]
lebensmittel = ["51000", "45341"]
kfz = ["45214"]
gastgewerbe = ["45213"]
verbraucherpreise = ["61351", "61111", "61241"]
konsumklima = ["52411", "61241", "61111", "61311", "47415", "52111", "52911", "91211", "99911"]
sonstiges = ["91111", "31111", "32121", "42231", "43531", "81000"]

importantTables = konsumausgaben + einkommen + einzelhandel + lebensmittel + kfz + gastgewerbe + verbraucherpreise + konsumklima + sonstiges
importantTables

In [ ]:
# Remove duplicates from list
importantTables = list(set(importantTables))

In [ ]:
# Apply filter
updatedTables = [x for x in updatedTables for y in importantTables if str(y) in x]

In [ ]:
# Where to download (folder name)
current_year = str(datetime.datetime.now().year)
destination = f"{current_year}/"
Path(destination).mkdir(parents=True, exist_ok=True)

In [ ]:
# Download XLSX-files for each updated Table Nr
tabUrl = "https://www-genesis.destatis.de/genesisWS/rest/2020/data/tablefile?username=" \
                +KENNUNG+"&password="+PASSWORT+"&format=csv&name=" # format: xlsx
# Get Table Title for each updated Table Nr
namUrl = "https://www-genesis.destatis.de/genesisWS/rest/2020/catalogue/tables?username=" \
                +KENNUNG+"&password="+PASSWORT+"&selection="

for name in updatedTables:
    # only download tables that have been updated since yesterday
    response = requests.get(tabUrl+name+"&stand="+yesterday, timeout=120)
    
    if response.status_code == 200:
        # a statistic may have been updated with values for June but tables with yearly data have not 
        if re.search("Keine aktualisierten Daten vorhanden.",str(response.content)):
            print(name, "Es liegen keine aktualisierten Daten vor.")

        else:     
            # enhance filename with human readable title 
            metaresp = requests.get(namUrl+name, timeout=120)
            # replace characters that are not suitable for filenames 
            filetitle = metaresp.json()["List"][0]["Content"].replace(":","_").replace("\n"," ").replace("/","_")
            # filename (statistics id + title) and directory
            with open(destination+name+"_"+filetitle+".csv", 'wb') as f:
                f.write(response.content)